# Seminar Python 3: Pandas - prelucrări statistice, gruparea și agregarea datalor în pachetul pandas

Conversia datelor din șir de caractere în dată calendaristică

In [ ]:
#Exemplu 1. Conversia coloanei date din șir de caractere în dată calendaristică
import dateutil
import pandas as pd
df = pd.read_csv('phone_data.csv')
print(df.dtypes)
df['date'] = df['date'].apply(dateutil.parser.parse, dayfirst=True)
print(df.dtypes)

Gruparea înregistrărilor și aplicarea funcțiilor pe grupuri

In [ ]:
#Exemplu 2. Posibilități de grupare
import pandas as pd
df = pd.read_csv('phone_data.csv')
print(df.groupby(['item']).groups.keys())
print(len(df.groupby(['item']).groups['call']))

import pandas as pd
df = pd.read_csv('phone_data.csv')
print(df.groupby(['month']).groups.keys())
print(len(df.groupby(['month']).groups['2014-11']))

In [ ]:
#Exemplu 3. utilizare max(), min(), mean(), first(), last() 
#la nivel de grup
import pandas as pd
df = pd.read_csv('phone_data.csv')
print('Prima inregistrare din coloana item pe valori distincte')
print(df.groupby('item').first())
print('Durata insumata pentru fiecare luna')
print(df.groupby('month')['duration'].sum())
print('Durata insumata pe convorbiri (calls), pentru fiecare retea')
print(df[df['item'] == 'call'].groupby('network')['duration'].sum())

In [ ]:
#Exemplu 4. Grupări complexe
import pandas as pd
df = pd.read_csv('phone_data.csv')
print('Numarul de apeluri, sms, transfer date pentru fiecare luna')
print(df.groupby(['month', 'item'])['date'].count())

In [ ]:
#Exemplu 5. Gruparea și agregarea datelor
import pandas as pd
df = pd.read_csv('phone_data.csv')
# Grupeaza dupa month si item si calculeaza statistici pentru fiecare grup
print(df.groupby(['month', 'item']).agg({'duration':sum,      # calculeaza suma duratelor dupa fiecare grup
                                     'network_type': "count", # numarul de tipuri de retele
                                     'date': 'first'}))    # arata prima aparitie (data) pentru fiecare grup
df1 = df.groupby(['month', 'item']).agg({'duration':sum,      # calculeaza suma duratelor dupa fiecare grup
                                     'network_type': "count", # numarul de tipuri de retele
                                     'date': 'first'})
df1.to_csv('agregare.csv')

In [ ]:
#Exemplu 6. Aplicarea unor funcții multiple unei singure coloane din grup
import pandas as pd
df = pd.read_csv('phone_data.csv')
# Grupeaza dupa month si item. Calculeaza statistici pentru fiecare grup
print(df.groupby(['month', 'item']).agg({'duration': [min, max, sum],      # calculeaza min, max, si sum pentru duration
                                     'network_type': "count", # calculeaza numarul de network_type
                                     'date': [min, 'first', 'nunique']}))    # calculeaza min, first, si numarul de date unice per grup

Prelucrarea seturilor de date cu merge / join dataframes

Vom utiliza fișierele **user_device.csv** și **user_usage.csv** având o coloană comună **use_id**.

Coloana **Model** din **supported_devices.csv** are valori comune cu  **device** din **user_device.csv**.

In [ ]:
#Exemplu 7. Inner merge / inner join
import pandas as pd
df = pd.read_csv('user_usage.csv')
df1 = pd.read_csv('user_device.csv')

result = pd.merge(df,
                  df1[['use_id', 'platform', 'device']],
                  on='use_id')
print(result)
print('Structura fisier user_usage.csv ', df.shape)
print('Structura fisier user_device.csv ', df1.shape)
print(df['use_id'].isin(df1['use_id']).value_counts())

In [ ]:
#Exemplu 8. Left merge / left join
import pandas as pd
df = pd.read_csv('user_usage.csv')
df1 = pd.read_csv('user_device.csv')

result = pd.merge(df,
                  df1[['use_id', 'platform', 'device']],
                  on='use_id',
                  how='left')
print(result)
print('Structura fisier user_usage.csv ', df.shape)
print('Structura fisier user_device.csv ', df1.shape)
print(df['use_id'].isin(df1['use_id']).value_counts())

In [ ]:
#Exemplu 9. Right merge / right join
import pandas as pd
df = pd.read_csv('user_usage.csv')
df1 = pd.read_csv('user_device.csv')

result = pd.merge(df,
                  df1[['use_id', 'platform', 'device']],
                  on='use_id',
                  how='right')
print(result)
print('Structura fisier user_usage.csv ', df.shape)
print('Structura fisier user_device.csv ', df1.shape)
print(df['use_id'].isin(df1['use_id']).value_counts())

In [ ]:
#Exemplu 10. Full outer merge / full outer join
import pandas as pd
df = pd.read_csv('user_usage.csv')
df1 = pd.read_csv('user_device.csv')

result = pd.merge(df,
                  df1[['use_id', 'platform', 'device']],
                  on='use_id',
                  how='outer')
print(result)
print('Structura fisier user_usage.csv ', df.shape)
print('Structura fisier user_device.csv ', df1.shape)
print(df['use_id'].isin(df1['use_id']).value_counts())

In [ ]:
#Exemplu 11. Full outer merge sau full outer join cu indicația _merge
import pandas as pd
df = pd.read_csv('user_usage.csv')
df1 = pd.read_csv('user_device.csv')

result = pd.merge(df,
                  df1[['use_id', 'platform', 'device']],
                  on='use_id',
                  how='outer',
                  indicator=True)
print(result)
print('Structura fisier user_usage.csv ', df.shape)
print('Structura fisier user_device.csv ', df1.shape)
print(df['use_id'].isin(df1['use_id']).value_counts())

In [ ]:
#Exemplu 12. Merge utilizând trei seturi de date

import pandas as pd
df = pd.read_csv('user_usage.csv')
df1 = pd.read_csv('user_device.csv')
df3 = pd.read_csv('supported_devices.csv')

result = pd.merge(df,
                  df1[['use_id', 'platform', 'device']],
                  on='use_id',
                  how='left')

df3.rename(columns={"Retail Branding": "manufacturer"}, inplace=True)
result = pd.merge(result,
                  df3[['manufacturer', 'Model']],
                  left_on='device',
                  right_on='Model',
                  how='left')
print(result.head())
print(result.shape)

In [ ]:
#Exemplu 13. Groupby și agg utilizând setul rezultat
import pandas as pd
df = pd.read_csv('user_usage.csv')
df1 = pd.read_csv('user_device.csv')
df3 = pd.read_csv('supported_devices.csv')

result = pd.merge(df,
                  df1[['use_id', 'platform', 'device']],
                  on='use_id',
                  how='left')

df3.rename(columns={"Retail Branding": "manufacturer"}, inplace=True)
result = pd.merge(result,
                  df3[['manufacturer', 'Model']],
                  left_on='device',
                  right_on='Model',
                  how='left')
print(result.groupby("manufacturer").agg({
    "outgoing_mins_per_month": "mean",
    "outgoing_sms_per_month": "mean",
    "monthly_mb": "mean",
    "use_id": "count"
}))

## Reprezentare grafică cu pachetul *matplotlib*

In [ ]:
#Exemplu 14. Reprezentare grafică cu bare cu matplotlib.pyplot
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option("display.max_columns",10)
df = pd.read_csv('clienti_leasing20.csv')
print(df['AGE'])
df['AGE'].plot(kind='bar')
plt.xlabel('ID_CLIENT')
plt.ylabel('AGE')
plt.show()

In [ ]:
#Exemplu 15. Reprezentare grafică cu bare, aplicarea de culori 
#în funcție de valoare (% din max.)
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
pd.set_option("display.max_columns",10)
df = pd.read_csv('clienti_leasing20.csv')
print(df['AGE'])
#all_colors = list(plt.cm.colors.cnames.keys())
colors = cm.hsv( df['AGE']/ float(max(df['AGE'])))
df['AGE'].plot(kind='bar',color=colors)
plt.xlabel('ID_CLIENT')
plt.ylabel('AGE')
plt.show()

In [ ]:
#Exemplu 16. Histrogramă cu matplotlib.pyplot
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option("display.max_columns",10)
df = pd.read_csv('clienti_leasing20.csv')
print(df['AGE'])
df['AGE'].plot(kind='hist')
plt.xlabel('AGE')
plt.show()

In [ ]:
#Exemplu 17. Grafic cu gruparea și sortarea datelor cu matplotlib.pyplot
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('clienti_leasing20.csv')
plot_data=df[df['SEX']=='m']
plot_data=plot_data.groupby('JOB')['INCOME_PER_YEAR'].sum()
plot_data.sort_values().plot(kind='bar')
plt.show()

# Referințe
J. VanderPlas, Python Data Science Handbook: https://jakevdp.github.io/PythonDataScienceHandbook/index.html  
https://pandas.pydata.org/docs/user_guide/index.html  
https://matplotlib.org/index.html  
https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/  
https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/

## Exerciții
1) Să se reprezinte grafic (de tip pie) durata însumată a sms pe lună, din **phone_data.csv**.

2) Să se creeze setul format din **user_usage** și **supported_devices** și să se reprezinte grafic (bare verticale) traficul însumat (coloana **monthly_mb**) pentru fiecare brand (coloana **Retail Branding**).

3) Să se afișeze, utilizând fișierul **phone_data.csv**, durata însumată pentru fiecare lună și durata însumată pentru un anumit tip de rețea (**mobile**) pentru fiecare lună.